In [1]:
import numpy as np
import os
import sys
import StringIO

In [17]:
# Function definitions (based on similar ones in camp2ascii code)
BIGENDIAN = False

tunits = {"HOUR": 3600., "MIN": 60., "SEC": 1., "MSEC": 1.e-3, "USEC": 1.e-6, "NSEC": 1.e-9}
subtunits = {"SecMsec": 1.e-3, "Sec100Usec": 100.e-6, "Sec10Usec": 10.e-6, "SecUsec": 1.e-6}


def decode_int2swap(val):
    # return val[0] << 8 + val[1]
    val0 = ord(val[0])
    val1 = ord(val[1])
    return (val0 << 8) + val1

def decode_fp2(val):
# C-code below
# 	SIresult=(int16_t)ReadInt2Swap(frame);
# 	sign     = ((0x8000 & SIresult) >> 15 );
# 	exponent = ((0x6000 & SIresult) >> 13 );
# 	mantissa = ((0x1FFF & SIresult)       );
# 	switch(exponent) {
# 		case 0: Fresult=(float)mantissa;
# 		break;
# 		case 1: Fresult=(float)mantissa*1e-1;
# 		break;
# 		case 2: Fresult=(float)mantissa*1e-2;
# 		break;
# 		default: Fresult=(float)mantissa*1e-3;
# 	}
# 	if(Fresult>=frame->fp2_nan || isnan(Fresult))
# 		fprintf(config->output,config->NANs);
# 	else {
# 		if(sign ==0)
# 			fprintf(config->output,config->fp2_format,Fresult);
# 		else
# 			fprintf(config->output,config->fp2_format,-Fresult);
# 	}
    SIresult = decode_int2swap(val)
    print "SIresult = ", SIresult
    sign = ((0x8000 & SIresult) >> 15 )
    print "sign = ", sign
    exponent = ((0x6000 & SIresult) >> 13 )
    print "exponent = ", exponent
    mantissa = ((0x1FFF & SIresult)       )
    print "mantissa = ", mantissa
    if exponent == 0:
        result = np.float(mantissa)
    elif exponent == 1:
        result = np.float(mantissa)*1.e-1
    elif exponent == 2:
        result = np.float(mantissa)*1.e-2
    elif exponent == 3:
        result = np.float(mantissa)*1.e-3
    
    if sign == 0:
        return result
    else:
        return -result
        
def read_onesec_file(path):
    frame_list_1s = []
    with open(path, 'rb') as osf:
        header1 = osf.readline()
        header2 = osf.readline()
        header2s = header2.strip().split(',')
        print header2s
        temp = header2s[1].split()
        notsp_res = int(temp[0].strip('""'))
        notsp_unit = str(temp[1].strip('"'))
        multiplier = 1.
        for t, m in tunits.iteritems():
            if t in notsp_unit:
                multiplier = m
                break
        print notsp_res, notsp_unit
        print multiplier
        frame_size = int(header2s[2].strip('""'))
        print frame_size
        subframe_res_str = header2s[5].strip('"')
        for t, m in subtunits.iteritems():
            if t in subframe_res_str:
                subframe_res = m
        print subframe_res
        header3 = osf.readline()
        header4 = osf.readline()
        header5 = osf.readline()
        header6 = osf.readline()
        print header6
        while True:
            frame = osf.read(frame_size)
            if frame == '':
                break
            else:
                print "We are here: ", osf.tell()
                print frame[:28]
                # Decode part of the frame (look at record number first)
                frame_header = np.frombuffer(frame, dtype=np.int32, count=3)
                print frame_header
                #print vals
                val1 = decode_fp2(frame[frame_header_size:frame_header_size+2])
                print val1
                frame_list_1s.append(frame)

    #     vals = np.frombuffer(frame1[frame_header_size:], dtype=np.int16, count=7)
    #     print vals
    #     for val in vals:
    #         val2 = np.frombuffer(val, dtype=np.float16)
    #         print val2
        # print vals

def parse_timestamp(frame_header):
    time_offset = frame_header[0] + TO_EPOCH
    subseconds = frame_header[1]
    timestamp = time_offset + subseconds
    print "{:x}".format(timestamp - TO_EPOCH)

In [4]:
# Parsivel ID
parsivel_id = "390654"

# Stuff related to the TOB3 data format
frame_header_size = 12
frame_footer_size = 4
data_size_10s_file = 5004
data_size_10s_actual = 4174 # Actual length of Parsivel string
frame_size_10s_no_padding = frame_header_size + data_size_10s_actual + frame_footer_size
print "10-s frame size (no padding): ", frame_size_10s_no_padding
frame_size_10s = frame_header_size + data_size_10s_file + frame_footer_size
print "10-s frame size (with padding): ", frame_size_10s

data_size_1s_file = 860 # I think
data_size_1s_actual = 172 #
frame_size_1s = frame_header_size + data_size_1s_file + frame_footer_size
frame_size_1s_no_padding = frame_header_size + data_size_1s_actual + frame_footer_size
nb_data_lines = np.int(np.floor(data_size_1s_file / data_size_1s_actual))


print "1-s frame size (no padding): ", frame_size_1s_no_padding
print "1-s frame size (with padding): ", frame_size_1s
print "1-s number of data lines in a frame: ", nb_data_lines 

10-s frame size (no padding):  4190
10-s frame size (with padding):  5020
1-s frame size (no padding):  188
1-s frame size (with padding):  876
1-s number of data lines in a frame:  5


In [5]:
datadir = '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming/'
outputdir = '/Users/dawson29/Dropbox/Projects/TriPIPS/fixed_incoming/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [6]:
onesecfilename = 'Met_Location_OneHz4231.dat'
tensecfilename = 'Parsivel_TenHz4231.dat' # 'Parsivel_TenHz4351.dat' # 'Parsivel_TenHz4346.dat'
garbledfilename = 'Met_Location_OneHz4351.dat'
correctedfilename = 'Met_Location_OneHz4351_corrected.dat'

onesecfilepath = os.path.join(datadir, onesecfilename)
tensecfilepath = os.path.join(datadir, tensecfilename)
garbledfilepath = os.path.join(datadir, garbledfilename)
correctedfilepath = os.path.join(datadir, correctedfilename)

In [18]:
read_onesec_file(onesecfilepath)

['"One_Hz"', '"1 SEC"', '"876"', '"330"', '"37940"', '"Sec100Usec"', '"           0"', '"           0"', '"2194352518"']
1 SEC
1.0
876
0.0001
"FP2","FP2","FP2","FP2","FP2","FP2","FP2","IEEE4B","IEEE4B","ASCII(24)","ASCII(24)","ASCII(24)","ASCII(24)","FP2","FP2","ASCII(24)","ASCII(24)","FP2"                                                                                                                                                                                                                                

We are here:  1900
A6    �S EIX&�g&` H�#�B�
[907675201         0   1004489]
SIresult =  17680
sign =  0
exponent =  2
mantissa =  1296
12.96
We are here:  2776
F6    �S EIX(Hi�` H�#�B�
[907675206         0   1004494]
SIresult =  17680
sign =  0
exponent =  2
mantissa =  1296
12.96
We are here:  3652
K6    �S EIX&�gv` H�#�B�
[907675211         0   1004499]
SIresult =  17668
sign =  0
exponent =  2
mantissa =  1284
12.84
We are here:  4528
P6    �S EIX&�g�` H�#�B�


In [ ]:
TO_EPOCH=631152000

frame_header = np.frombuffer(frame1, dtype=np.int32, count=3)
print frame_header
time_offset = frame_header[0] + TO_EPOCH
subseconds = frame_header[1]
timestamp = time_offset + subseconds
print "{:x}".format(timestamp - TO_EPOCH)

# Ok, looks like I'm reading this correctly - it matches the output I get from camp2ascii

In [ ]:
with open(tensecfilepath, 'rb') as osf:
    header1 = osf.readline()
    header2 = osf.readline()
    header2 = header2.strip().split(',')
    print header2
    frame_size_10s = int(header2[2].strip('""'))
    print frame_size_10s
    header3 = osf.readline()
    header4 = osf.readline()
    header5 = osf.readline()
    header6 = osf.readline()
    print header6
    frame = osf.read(frame_size_10s_no_padding)
    print frame
    print len(frame)

In [ ]:
frame_list = []
# Now open the garbled file
with open(garbledfilepath, 'rb') as osf:
    header1 = osf.readline()
    header2 = osf.readline()
    header2s = header2.strip().split(',')
    print header2s
    #frame_size = int(header2[2].strip('""'))
    #print frame_size
    header3 = osf.readline()
    header4 = osf.readline()
    header5 = osf.readline()
    header6 = osf.readline()
    print header6
# Look for the Parsivel ID in the file
#     data = osf.read()
#     data.find(parsivel_id)
#     location = osf.tell()
#     osf.seek(location - frame_header_size)
    data_start = osf.tell() # Start of data in file
    data = osf.read() # Read in all the data
    loc = data.find(parsivel_id)
    # Find all locations of the parsivel_id and grab the dataframe associated with it.
    # This is pretty inefficient; can speed it up using mmap?
    while loc > -1:
        osf.seek(data_start + loc - frame_header_size, 0)
        #print "We are here: ", osf.tell()
        frame = osf.read(frame_size_10s_no_padding)
        #print frame
        #print len(frame)
        frame_list.append(frame)
        loc = data.find(parsivel_id, loc + 1)

# Now, write out a file with all of the Parsivel frames removed. Hopefully this fixes the problem
for frame in frame_list:
    data = data.replace(frame, "")
    
with open(correctedfilepath, 'wb') as cf:
    cf.write(header1)
    cf.write(header2)
    cf.write(header3)
    cf.write(header4)
    cf.write(header5)
    cf.write(header6)
    cf.write(data)
    
# Open the "corrected" file and take a look to see if we can see additional frames
frame_list2 = []
with open(correctedfilepath, 'rb') as cf:
    header1 = cf.readline()
    header2 = cf.readline()
    header2s = header2.strip().split(',')
    print header2s
    frame_size = int(header2s[2].strip('""'))
    print frame_size
    header3 = cf.readline()
    header4 = cf.readline()
    header5 = cf.readline()
    header6 = cf.readline()
    print header6
    while True:
        frame = cf.read(frame_size)
        if frame == '':
            break
        else:
            #print frame
            # Decode part of the frame
            vals = np.frombuffer(frame, dtype=np.float16, count=7)
            print vals
            print "We are here: ", cf.tell()
            frame_list2.append(frame)

In [ ]:
print frame_list[8]

In [ ]:
frame_header = np.frombuffer(frame3, dtype=np.int32, count=3)
print frame_header
time_offset = frame_header[0] + TO_EPOCH
subseconds = frame_header[1]
timestamp = time_offset + subseconds
print "{:x}".format(timestamp - TO_EPOCH)